# Вырезка нужных годов 2010-2015

In [1]:
START_YEAR = 2010
END_YEAR = 2015
BORDER_YEAR = 2014

In [8]:
fin = open('dblp.xml', 'r')
fout_train = open('train.xml', 'w')
fout_test = open('test.xml', 'w')

article = []
is_train = False
is_test = False

for line in fin:
    if '<article ' in line:
        article = []
        is_train = False
        is_test = False

    article.append(line)
    try:
        if '<year>' in line:
            year = line[line.find("<year>") + 6:line.find("</year>")]
            if START_YEAR <= int(year) <= BORDER_YEAR:
                is_train = True
            if BORDER_YEAR < int(year) <= END_YEAR:
                is_test = True
    except:
        print line
        break
    
    if '</article>' in line:
        if is_train:
            fout_train.write('\n'.join(article))
        if is_test:
            fout_test.write('\n'.join(article))

fout_train.close()
fout_test.close()
fin.close()

In [9]:
fin_train = open('train.xml', 'r')
fin_test = open('test.xml', 'r')

print fin_train.read().count('<article ')
print fin_test.read().count('<article ')

481128
113351


# Переводим в граф

In [10]:
Nodes = {}
next_free_id = 0

In [14]:
fin_train = open('train.xml', 'r')

for line in fin_train:
    if "<author>" in line:
        author = line[line.find("<author>") + 8:line.find("</author>")]
        if author not in Nodes:
            Nodes[author] = next_free_id
            next_free_id += 1
    if line.count('<author>') > 1:
        print 'Err'
fin_train.close()

In [15]:
next_free_id

546295

In [17]:
def convert_graph(fin, fout, Nodes):
    article = []
    authors = []
    edges = 0
    for line in fin:
        if '<article ' in line:
            article = []
            authors = []

        article.append(line)
        if "<author>" in line:
            author = line[line.find("<author>") + 8:line.find("</author>")]
            if author in Nodes:
                authors.append(Nodes[author])

        if '</article>' in line:
            for i in range(len(authors)):
                for j in range(i+1, len(authors)):
                    fout.write(str(authors[i]) + ' ' + str(authors[j]) + '\n')
                    edges += 1
    print 'edges', edges

In [18]:
fin_train = open('train.xml', 'r')
fout_train = open('train.in', 'w')
convert_graph(fin_train, fout_train, Nodes)
fout_train.close()
fin.close()

edges 2351026


In [19]:
fin_test = open('test.xml', 'r')
fout_test = open('test.out', 'w')
convert_graph(fin_test, fout_test, Nodes)
fin_test.close()
fout_test.close()

edges 370418


# Подсчет новых ребер

In [20]:
fin_train = open('train.in', 'r')
train_edges = set()
for line in fin_train:
    line = line.strip()
    train_edges.add(line)
fin_train.close()

In [23]:
fin_test = open('test.out', 'r')
number_new_edges = 0
for line in fin_test:
    u, w = line.strip().split()
    if (u + ' ' + w not in train_edges) and (
        w + ' ' + u not in train_edges):
        number_new_edges += 1
fin_test.close()
print 'number_new_edges', number_new_edges

number_new_edges 156306
